In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import scienceplots
import seaborn as sns
import matplotlib.animation as animation
from scipy.fft import fft, rfft, fftfreq, rfftfreq
from scipy import signal
from scipy.interpolate import interp1d

def get_pos(N, N_e=8):

    pos     = {}
    node    = N_e

    for y in range(N):
        for x in range(N):

            pos[node]   = (x,y)
            node        += 1

    pos[0] = (-1,-1)
    if N % 2 != 0:
        pos[1] = (int(N/2),-1)
    else:
        pos[1] = (int(N/2)-1,-1)
    pos[2] = (N,-1)
    if N % 2 != 0:
        pos[3] = (-1,int(N/2))
    else:
        pos[3] = (-1,int(N/2)-1)
    pos[4] = (-1,N)
    pos[5] = (N,int(N/2))
    pos[6] = (int(N/2),N)
    pos[7] = (N,N)

    return pos

def min_max(arr):
    return (arr - np.min(arr))/(np.max(arr) - np.min(arr))

N_voltages      = 10000
step_size       = 1e-8
time_steps      = np.cumsum(np.repeat(step_size, N_voltages))
sample_rate     = (N_voltages-1)/np.max(time_steps)
amplitude       = 0.2
time_steps      = time_steps[1:]
time_start      = 4000
time_steps      = time_steps[time_start:]
blue_color      = '#4477AA'
red_color       = '#EE6677'

N_NP        = 3
cmeter      = 1/2.54
single_col  = 8.6

frequencies = [1e5,2e5,3e5,4e5,5e5]

In [ ]:
def nonlinear_index(df : pd.DataFrame, main_frequency : float, sample_rate : float, time_scale=10**6, number_of_frequencies=10):

    N_values    = len(df)
    values      = []

    for col in df.columns:

        # Fourier Spectrum
        arr     = df.loc[:,col].values
        xf      = rfftfreq(N_values-1, 1 / sample_rate)/time_scale
        yf      = 2*np.abs(rfft(arr))/N_values
        df_f    = pd.DataFrame([xf, yf]).T
        df_f    = df_f[df_f[0] >= main_frequency/time_scale]
        df_f    = df_f.loc[df_f.loc[:,1].nlargest(number_of_frequencies).index].reset_index(drop=True)

        a       = df_f.iloc[1:,1].sum()
        b       = df_f.iloc[0,1]

        n_idx   = a/b
        values.append(n_idx)

    return values

## State

In [ ]:
input_values = {}

for frequency in frequencies:
    
    values                  = amplitude*np.cos(2*np.pi*frequency*time_steps)
    input_values[frequency] = values

In [ ]:
def load_periodic_data(N_NP, path="../../../data/periodic/", frequencies=[1e5,2e5,3e5,4e5,5e5]):

    data_states = {}

    for frequency in frequencies:

        dfs         = {}
        n_states    = 0

        for i in range(0,10):
            for j in range(5):

                df_tmp          = pd.read_csv(path+f"f={(np.round(frequency))}_A={amplitude}mean_state_Nx={N_NP}_Ny={N_NP}_Nz=1_Ne=2_{i}_{j}.csv")
                dfs[n_states]   = df_tmp.iloc[time_start:,:].reset_index(drop=True)
                n_states        += 1

        data_states[frequency]  = dfs

    return data_states

In [ ]:
data_states     = load_periodic_data(7)
data_states_dis = load_periodic_data(5, path="../../../data/periodic/dis_")

### Example Frequency Spectrum

In [ ]:
n_run       = 0
np_idx      = 22
frequency   = 1e5

with plt.style.context(['science', 'bright','grid']):

    fig = plt.figure()
    ax  = fig.add_subplot()
    ax.plot(time_steps*10**6, input_values[frequency]*1000, c=blue_color)

    ax2 = ax.twinx()
    ax2.plot(time_steps*10**6, data_states_dis[frequency][n_run].iloc[:,np_idx], c=red_color)

    ax.set_xlabel("$t$ [$\mu s$]")
    ax.set_ylabel("$U_I$ [$mV$]")
    # ax.set_xlim(45,50)
    # ax.set_ylim(-200,200)
    ax2.set_ylabel(f"$<n_{np_idx}>$ [$e$]")
    # ax2.set_ylim(-.005,.005)

plt.show()

In [ ]:
arr     = data_states[frequency][n_run].iloc[:,np_idx].values
# arr     = (arr - np.mean(arr))/np.std(arr)
xf      = rfftfreq(len(arr)-1, 1 / sample_rate)/10**6
yf      = 2*np.abs(rfft(arr))/len(arr)

with plt.style.context(['science', 'bright','grid']):

    fig = plt.figure()
    ax  = fig.add_subplot()
    ax.plot(xf, yf, label="$f_{NP}$", lw=1)
    ax.axvline(frequency/10**6, color=red_color, ls='--', label="$f_{Input}$")
    ax.set_xlabel("$f$ [$MHz$]")
    ax.set_yscale('log')
    ax.set_xlim(0,2)
    ax.set_ylim(10**-6)
    ax.legend()
    ax.set_ylabel('Amplitude [$e$]')

plt.show()

In [183]:
idx_vals1   = [nonlinear_index(data_states[frequency][n_run], frequency, sample_rate, number_of_frequencies=10) for frequency in frequencies]
idx_vals2   = [nonlinear_index(data_states_dis[frequency][n_run], frequency, sample_rate, number_of_frequencies=10) for frequency in frequencies]

/tmp/ipykernel_163276/802948480.py:19: RuntimeWarning: invalid value encountered in double_scalars
  n_idx   = a/b
/tmp/ipykernel_163276/802948480.py:19: RuntimeWarning: invalid value encountered in double_scalars
  n_idx   = a/b


In [186]:
np.nanmean(idx_vals1)

4.999001043660096

In [185]:
np.nanmean(idx_vals2)

4.843593968724979

## NP Phase Space

In [ ]:
ele_charge = 1.60217663e-19
plot_order = [20,21,22,23,24,15,16,17,18,19,10,11,12,13,14,5,6,7,8,9,0,1,2,3,4]

with plt.style.context(['science', 'bright','grid']):

    fig = plt.figure()

    for n,i in enumerate(plot_order):
        
        ax  = fig.add_subplot(5,5,n+1)
        ax.plot(df.loc[4000:5000,'E0']*1000, 1000*df_states.iloc[4000:5000,i])
        ax.set_xlim(-200,200)
        ax.set_ylim(-4,4)
        
        ax.set_xticklabels('')
        ax.set_yticklabels('')

        if n > 19:
            ax.set_xticks([-150,0,150])
            ax.set_xticklabels([-150,0,150], fontsize='small')

        if n % 5 == 0:
            ax.set_yticks([-3,0,3])
            ax.set_yticklabels([-3,0,3], fontsize='small')

    fig.supxlabel('$U_I$ [$mV$]', y=-0.05)
    fig.supylabel('$<n>$ [$\\frac{e}{1000}$]')

    fig.savefig(f"plots/states/"+f"f={(np.round(frequency))}_A={amplitude}"+f"N_{N_NP}_n_vs_U_grid.svg", bbox_inches='tight', transparent=True)
    fig.savefig(f"plots/states/"+f"f={(np.round(frequency))}_A={amplitude}"+f"N_{N_NP}_n_vs_U_grid.pdf", bbox_inches='tight', transparent=False)

    plt.show()

## Disorder

In [ ]:
N_voltages  = 10000
step_size   = 1e-8
time_steps  = np.cumsum(np.repeat(step_size, N_voltages))
sample_rate = (N_voltages-1)/np.max(time_steps)
amplitude   = 0.2
time_steps  = time_steps[1:]
N_NP        = 7

In [ ]:
data    = {}
N_NP    = 7

for frequency in [1e5,2e5,3e5,4e5,5e5]:

    dfs     = []
    n_stat  = 0

    for i in range(0,10):
        for j in range(5):

            df_tmp      = pd.read_csv(f"../../../data/periodic/f={frequency}_A={amplitude}Nx={N_NP}_Ny={N_NP}_Nz=1_Ne=2_{i}_{j}.csv")
            dfs.append(df_tmp)
            n_stat      += 1

    df              = sum(dfs)/len(dfs)
    test            = pd.DataFrame(np.std(np.stack(dfs), axis=0), columns=df.columns)
    df['Error']     = test['Current']
    df              = df.drop(columns=['Eq_Jumps'])
    df['Error']     = df['Error']/np.sqrt(n_stat)
    data[frequency] = df

In [ ]:
pos = [(-1,-1),(5,5),(0,0),(1,0),(2,0),(3,0),(4,0),
       (0,1),(1,1),(2,1),(3,1),(4,1),
       (0,2),(1,2),(2,2),(3,2),(4,2),
       (0,3),(1,3),(2,3),(3,3),(4,3),
       (0,4),(1,4),(2,4),(3,4),(4,4)]
pos = {i : pos[i] for i in range(0,27)}

In [ ]:
N_NP    = 5
G       = nx.DiGraph()
G.add_nodes_from([i for i in range(N_NP*N_NP+2)])
G.add_edges_from([eval(i) for i in pd.read_csv(f"../../../data/periodic/dis_f={1e5}_A={amplitude}net_currents_Nx={N_NP}_Ny={N_NP}_Nz=1_Ne=2_0_0.csv").columns])
node_color      = np.repeat(blue_color, N_NP*N_NP+2)
node_color[0:2] = None
node_labels     = {i:'' for i in range(N_NP*N_NP+2)}
node_labels[0]  = '$I$'
node_labels[1]  = '$O$'

In [ ]:
with plt.style.context(['science','bright']):
    
    fig         = plt.figure()
    ax1         = fig.add_subplot()
    sizes       = np.repeat(0,N_NP*N_NP+2)
    sizes[2:]   = np.repeat(100,N_NP*N_NP)
    node_colors = np.repeat(blue_color, N_NP*N_NP+2)

    nx.draw_networkx(G, pos=pos, node_size=sizes, ax=ax1, node_color=node_colors, arrows=False, labels=node_labels, font_size=20)
    plt.tight_layout()
    plt.axis('off')
    fig.savefig(f"plots/graphs/default_graph_{N_NP}.svg", bbox_inches='tight', transparent=True)
    plt.show()

In [ ]:
with plt.style.context(['science','bright']):
    
    fig         = plt.figure()
    ax1         = fig.add_subplot()
    sizes       = np.repeat(0,N_NP*N_NP+2)
    sizes[2:]   = np.abs(np.random.normal(100,50,N_NP*N_NP))
    node_colors = np.repeat(blue_color, N_NP*N_NP+2)

    nx.draw_networkx(G, pos=pos, node_size=sizes, ax=ax1, node_color=node_colors, arrows=False, labels=node_labels, font_size=20)
    plt.tight_layout()
    plt.axis('off')
    fig.savefig(f"plots/graphs/default_dis_graph_{N_NP}.svg", bbox_inches='tight', transparent=True)
    plt.show()

In [ ]:
data_dis    = {}
N_NP        = 5

for frequency in [1e5,2e5,3e5,4e5,5e5]:

    dfs     = []
    n_stat  = 0

    for i in range(0,10):
        for j in range(5):

            df_tmp      = pd.read_csv(f"../../../data/periodic/dis_f={frequency}_A={amplitude}Nx={N_NP}_Ny={N_NP}_Nz=1_Ne=2_{i}_{j}.csv")
            dfs.append(df_tmp)
            n_stat      += 1

    df                  = sum(dfs)/len(dfs)
    test                = pd.DataFrame(np.std(np.stack(dfs), axis=0), columns=df.columns)
    df['Error']         = test['Current']
    df                  = df.drop(columns=['Eq_Jumps'])
    df['Error']         = df['Error']/np.sqrt(n_stat)
    data_dis[frequency] = df

In [ ]:
plt.plot(data[1e5]['Current'])
plt.plot(data_dis[1e5]['Current'])
plt.xlim(8000,10000)
plt.ylim(-5000,5000)
plt.show()

In [ ]:
df_tmp      = pd.DataFrame()
df_tmp['U'] = (df.loc[1000:,'E0']*1000).values
df_tmp['n'] = (df_states.iloc[1000:,24]).values
df_tmp['t'] = (time_steps*10**6)[1000:]
df_tmp

In [ ]:
sns.scatterplot(data=df_tmp, x='U', y='n', hue='t', palette='magma')
plt.show()

In [ ]:
ele_charge  = 0.160217662
cmap        = plt.get_cmap('magma')

with plt.style.context(['science', 'bright']):

    fig = plt.figure()
    ax  = fig.add_subplot()
    ax.plot(df.loc[1000:,'E0']*1000, df_states.iloc[1000:,0], '-')
    # ax.set_xlim(0,1000)
    ax.set_ylim()

# plt.plot(df_states.iloc[:,10])
plt.show()

## I-O dependency

In [ ]:
with plt.style.context(['science', 'bright', 'notebook']):
    
    fig = plt.figure()
    ax1 = fig.add_subplot()
    ax1.plot(time_steps*10**6, min_max(df['E0']), '-o', color=blue_color, label='Input')
    ax1.plot(time_steps*10**6, min_max(df['Current']), '-s', color=red_color, label='Output')
    # ax1.fill_between(time_steps*10**6, df['Current']-df['Error'], df['Current']+df['Error'], color=red_color, alpha=.6)
    ax1.set_xlabel('$t$ [$\mu s$]')
    ax1.set_ylabel('Signal')
    ax1.set_xlim(0,500)
    ax1.legend(loc='upper left')
    # ax1.set_ylim(-1,1)

    # fig.savefig(f"plots/"+f_str+"I_O_periodic_signal.svg", bbox_inches='tight', transparent=True)
    # fig.savefig(f"plots/"+f_str+"I_O_periodic_signal.pdf", bbox_inches='tight', transparent=False)
plt.show()

## combined

In [ ]:
signal  = np.int16((df['Current'].values / df['Current'].values.max()) * 32767)
signal  = df['Current'].values
input_f = freq*(1e5)/10**6

with plt.style.context(['science', 'bright', 'notebook']):

    fig = plt.figure()
    fig.set_figwidth(2.5*single_col*cmeter)
    fig.set_figheight(single_col*cmeter/1)

    ax1 = fig.add_subplot(1,2,1)
    ax2 = ax1.twinx()
    ax1.plot(time_steps*10**6, df['Current'].values, '-', color=blue_color, label='$I$')
    ax1.fill_between(time_steps*10**6, df['Current']-df['Error'], df['Current']+df['Error'], color=blue_color, alpha=.6)
    ax2.plot(time_steps*10**6, df['E0'].values, '--', color=red_color, label='$U_I$')
    ax1.set_xlabel('$t$ [$\mu s$]')
    # ax1.set_ylabel('Signal')
    ax1.set_xlim(0,500)
    ax1.set_yticklabels("")
    ax2.set_yticklabels("")
    # ax1.legend(loc='lower right')

    yf = rfft(signal)
    xf = rfftfreq(N_voltages-1, 1 / sample_rate)

    ax  = fig.add_subplot(1,2,2)
    ax.plot(xf/10**6, np.abs(yf), label="$\omega_{Output}$")
    ax.axvline(input_f, color=red_color, label="$\omega_{Input}$", ls='--')
    ax.set_xlabel("$\omega$ [$\\frac{1}{\mu s}$]")
    ax.set_ylim(0)
    ax.set_xlim(0,0.2)
    ax.legend()
    ax.set_yticklabels("")
    plt.subplots_adjust(wspace=0.2)

    fig.savefig(f"plots/"+f_str+f"N={N_NP}_1_In_1_Out_time_dep.svg", bbox_inches='tight', transparent=True)
    fig.savefig(f"plots/"+f_str+f"N={N_NP}_1_In_1_Out_time_dep.pdf", bbox_inches='tight', transparent=False)

plt.show()

## Animtaion

In [ ]:
G   = nx.DiGraph()
G.add_nodes_from([i for i in range(N_NP*N_NP+8)])
G.add_edges_from([eval(i) for i in pd.read_csv(f"data/periodic/"+f_str+f"net_currents_Nx={N_NP}_Ny={N_NP}_Nz=1_Ne=8_0_0.csv").columns])
pos             = get_pos(N_NP)
node_color      = np.repeat(blue_color, N_NP*N_NP+8)
node_color[0:8] = None
node_labels     = {i:'' for i in range(N_NP*N_NP+8)}
node_labels[0]  = '$I$'
node_labels[2]  = '$C$'
node_labels[4]  = '$C$'
node_labels[7]  = '$O$'
node_labels[1]  = '$C$'
node_labels[3]  = '$C$'
node_labels[5]  = '$C$'
node_labels[6]  = '$C$'

In [ ]:
n_stat  = 1
states  = pd.read_csv(f"data/periodic/"+f_str+"mean_state_Nx=7_Ny=7_Nz=1_Ne=8_0_0.csv")

for i in range(0,10):
    for j in range(5):
        if not((i == 0) and (j == 0)):
            states += pd.read_csv("data/periodic/"+f_str+f"mean_state_Nx=7_Ny=7_Nz=1_Ne=8_{i}_{j}.csv")
            n_stat += 1
            
states = states/n_stat

In [ ]:
raw_data    = pd.read_csv("data/periodic/"+f_str+"Nx=7_Ny=7_Nz=1_Ne=8_0_0.csv")
in_vals     = 1000*raw_data['E0'].values
t_vals      = time_steps*10**6

start_ind   = 1500
n_frames    = 500
in_vals     = in_vals[start_ind:start_ind+n_frames]
t_vals      = t_vals[start_ind:start_ind+n_frames]

with plt.style.context(['science','bright']):
    
    fig = plt.figure()
    fig.set_figwidth(2.5*single_col*cmeter)
    fig.set_figheight(single_col*cmeter/1)
    ax1 = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)

    def update(row):

        ax1.clear()
        ax1.plot(t_vals, in_vals)
        ax1.plot(t_vals[row], in_vals[row], 'o')
        ax1.set_xlabel('$t$ [$\mu s$]')
        ax1.set_ylabel('Input [$mV$]')
        ax1.set_xlim(np.round(np.min(t_vals)),np.round(np.max(t_vals)))
        ax1.set_ylim(-100,100)

        ax2.clear()

        sizes       = np.repeat(0,49+8)
        s_vals      = states.iloc[row].values
        sizes[8:]   = 5000*np.abs(s_vals)

        node_colors = np.repeat(blue_color, 49+8)
        node_colors[8:][s_vals < 0] = red_color

        nx.draw_networkx(G, pos=pos, node_size=sizes, ax=ax2, node_color=node_colors, arrows=False, labels=node_labels, font_size=16)
        ax.set_title(f'$t = {np.round((10**6)*time_steps[row],2)}$ $\mu s$')
        plt.tight_layout()
        # plt.axis('off')
        
    ani = animation.FuncAnimation(fig, update, frames=n_frames, interval=200, repeat_delay=1000, repeat=True)
    ani.save("plots/gifs/"+f_str+"anim.gif", dpi=200)

plt.show()

In [ ]:
with plt.style.context(['science','bright']):
    
    fig         = plt.figure()
    ax1         = fig.add_subplot()
    sizes       = np.repeat(0,N_NP*N_NP+8)
    sizes[8:]   = np.repeat(100,N_NP*N_NP)
    node_colors = np.repeat(blue_color, N_NP*N_NP+8)

    nx.draw_networkx(G, pos=pos, node_size=sizes, ax=ax1, node_color=node_colors, arrows=False, labels=node_labels, font_size=20)
    plt.tight_layout()
    plt.axis('off')
    fig.savefig(f"default_graph_{N_NP}.svg", bbox_inches='tight', transparent=True)
    plt.show()

### Noise

In [ ]:
noise_data  = {}
n           = 0

for i in range(0,10):

    for j in range(5):

        df_tmp          = pd.read_csv(f"data/noise/Nx=5_Ny=5_Nz=1_Ne=8_{i}_{j}.csv")
        df_tmp          = df_tmp.drop(columns=['Eq_Jumps', 'Error'])
        noise_data[n]   = df_tmp
        n               += 1

noise_data[0]

In [ ]:
n = 20

with plt.style.context(['science', 'bright', 'grid']):
    
    fig = plt.figure()
    ax1 = fig.add_subplot()
    ax1.plot(time_steps*10**6, noise_data[n]['E0']*1000, '.-', color=blue_color, markevery=10)
    # ax1.plot(time_steps*10**6, noise_data[10]['E0'], '.-', color=blue_color, markevery=10)
    # ax1.fill_between(time_steps*10**6, df['Current']-df['Error'], df['Current']+df['Error'], color=red_color, alpha=.6)
    ax1.set_xlabel('$t$ [$\mu s$]')
    ax1.set_ylabel('$U_I$ [mV]')
    ax1.set_xlim(0,1000)
    # ax1.legend(loc='upper left')
    # ax1.set_ylim(-1,1)

    fig.savefig(f"plots/noise_input_signal_{n}.svg", bbox_inches='tight', transparent=True)
    fig.savefig(f"plots/noise_input_signal_{n}.pdf", bbox_inches='tight', transparent=False)

plt.show()

In [ ]:
n = 20

with plt.style.context(['science', 'bright', 'grid']):
    
    fig = plt.figure()
    ax1 = fig.add_subplot()
    ax1.plot(time_steps*10**6, noise_data[n]['Current'], '.-', color=blue_color, markevery=10)
    # ax1.plot(time_steps*10**6, noise_data[10]['E0'], '.-', color=blue_color, markevery=10)
    # ax1.fill_between(time_steps*10**6, df['Current']-df['Error'], df['Current']+df['Error'], color=red_color, alpha=.6)
    ax1.set_xlabel('$t$ [$\mu s$]')
    ax1.set_ylabel('$I_O$ [$pA$]')
    ax1.set_xlim(0,1000)
    # ax1.legend(loc='upper left')
    # ax1.set_ylim(-1,1)

    fig.savefig(f"plots/noise_output_signal_{n}.svg", bbox_inches='tight', transparent=True)
    fig.savefig(f"plots/noise_output_signal_{n}.pdf", bbox_inches='tight', transparent=False)

plt.show()

In [ ]:
fig = plt.figure()
ax  = fig.add_subplot()

for key,df in noise_data.items():

    ax = df['Current'].plot(ax=ax)

plt.show()

In [ ]:
def memory_cap(x, y):

    x_norm  = x - np.mean(x)
    y_norm  = y - np.mean(y)
    lags    = np.arange(0, len(test_x))
    
    np.roll(x, -lag)-np.mean(x)
    

In [ ]:
ccorr_list = []

for key, df in noise_data.items():
    
    test_x  = df['E0'].values
    test_y  = df['Current'].values
    lags    = np.arange(0, len(test_x))
    x       = test_x - np.mean(test_x)
    y       = test_y - np.mean(test_y)
    ccorr   = [(np.correlate(test_x, np.roll(test_y, -lag)) / (np.linalg.norm(test_x) * np.linalg.norm(test_y)))[0] for lag in lags]
    
    ccorr_list.append(ccorr)

In [ ]:
means   = np.mean(np.array(ccorr_list).T,axis=1)
stds    = np.std(np.array(ccorr_list).T,axis=1)

with plt.style.context(['science', 'bright','notebook']):

    fig = plt.figure()
    ax  = fig.add_subplot()
    ax.plot(time_steps*10**6, means, '-')
    ax.fill_between(time_steps*10**6, means-stds, means+stds, alpha=0.6)
    # ax.plot(time_steps*10**6, np.mean(np.array(ccorr_list).T,axis=1))
    ax.set_xlabel("$\\tau$ [$\mu s$]")
    ax.set_xlim(0,100)
    ax.set_ylim(0,1)
    plt.show()

In [ ]:
with plt.style.context(['science', 'bright','notebook']):

    fig = plt.figure()
    ax  = fig.add_subplot()
    ax.plot(np.arange(0,len(test_x)), test_x)
    ax2 = ax.twinx()
    ax2.plot(np.arange(len(test_x),len(test_x)*2), test_y)

In [ ]:
# Sample time series data for x and y
x = np.array([1, 2, 3, 4, 5])
y = np.array([3, 4, 5, 6, 7])

# Calculate the cross-correlation using the normalized cross-correlation formula
cross_corr = [np.correlate(x, np.roll(y, -lag)) / (np.linalg.norm(x) * np.linalg.norm(y)) for lag in range(len(x))]

# Create the lag values
lags = range(len(x))

# Plot the standardized cross-correlation as a function of τ
plt.plot(lags, cross_corr)
plt.xlabel('Time Lag (τ)')
plt.ylabel('Standardized Cross-correlation')
plt.show()

In [ ]:
blue_color          = '#4477AA'
red_color           = '#EE6677'

with plt.style.context(["science","nature","bright"]):
    
    fig, ax1    = plt.subplots(dpi=200)
    ax2         = ax1.twinx()

    ax1.errorbar(x=time_steps[:-1]*1000 ,y=df['O'], yerr=df['sO'], c=blue_color)
    ax2.plot(time_steps[:-1]*1000, df['I']*1000, c=red_color)
    ax1.set_xlabel("Time [ms]")
    ax1.set_ylabel("Output [pA]")
    ax2.set_ylabel("Input [mV]")

    ax1.legend(['Output'], loc='lower left')
    ax2.legend(['Input'], loc='lower right')

    ax1.set_ylim(-1500,1500)
    ax2.set_ylim(-50,50)

plt.show()

In [ ]:
with plt.style.context(["science","nature","bright"]):
    
    fig, ax1    = plt.subplots(dpi=200)
    df          = dic[0.5]

    ax1.plot(df['O'], df['I']*1000)
    ax1.set_xlabel("Input [mV]")
    ax1.set_ylabel("Output [pA]")
    plt.show()

# States

In [ ]:
G   = nx.DiGraph()
G.add_nodes_from([i for i in range(49+8)])
G.add_edges_from([eval(i) for i in pd.read_csv(f"data/periodic/net_currents_Nx=7_Ny=7_Nz=1_Ne=8_0_0.csv").columns])
pos             = get_pos(7)
node_color      = np.repeat(blue_color, 49+8)
node_color[0:8] = None
node_labels     = {i:'' for i in range(49+8)}
node_labels[0]  = '$I$'
node_labels[2]  = '$C$'
node_labels[4]  = '$C$'
node_labels[7]  = '$O$'
node_labels[1]  = '$C$'
node_labels[3]  = '$C$'
node_labels[5]  = '$C$'
node_labels[6]  = '$C$'

In [ ]:
states = pd.read_csv(f"data/periodic/mean_state_Nx=7_Ny=7_Nz=1_Ne=8_0_0.csv")
states

In [ ]:
s_vals = states.abs()
s_vals = s_vals/np.max(s_vals.max())
s_logs = np.log(s_vals)
s_logs = s_logs - np.min(s_logs.min())
s_logs

In [ ]:
s_flat  = states.values.flatten()
s_flat  = (s_flat - np.min(s_flat))/(np.max(s_flat) - np.min(s_flat))
s_flat  = s_flat.reshape(-1,49)
s_flat

In [ ]:
with plt.style.context(['science','bright']):
    
    fig = plt.figure()
    ax  = fig.add_subplot()

    def update(row):
        
        ax.clear()

        sizes       = np.repeat(0,49+8)
        s_vals      = states.iloc[row].values
        sizes[8:]   = 500*np.abs(s_vals)

        node_colors = np.repeat(blue_color, 49+8)
        node_colors[8:][s_vals < 0] = red_color

        nx.draw_networkx(G, pos=pos, node_size=sizes, ax=ax, node_color=node_colors, arrows=False, labels=node_labels, font_size=8)
        ax.set_title(f'$t = {np.round((10**6)*time_steps[row],2)}$ $\mu s$')
        plt.axis('off')


    ani = animation.FuncAnimation(fig, update, frames=len(states), interval=200, repeat_delay=1000, repeat=True)
    ani.save("test.gif")

In [ ]:
raw_data    = pd.read_csv(f"data/periodic/Nx=7_Ny=7_Nz=1_Ne=8_0_0.csv")
in_vals     = 1000*raw_data['E0'].values
t_vals      = time_steps*10**6

with plt.style.context(['science','bright']):
    
    fig = plt.figure()
    ax  = fig.add_subplot()

    def update(step):
        ax.clear()
        ax.plot(t_vals, in_vals)
        ax.plot(t_vals[step], in_vals[step], 'o')
        ax.set_xlabel('$t$ [$\mu s$]')
        ax.set_ylabel('Input [$mV$]')
        ax.set_xlim(0,200)
        ax.set_ylim(-50,50)
        
    ani = animation.FuncAnimation(fig, update, frames=len(states), interval=200, repeat_delay=1000, repeat=True)
    ani.save("test2.gif")

plt.show()

In [ ]:
raw_data    = pd.read_csv(f"data/periodic/Nx=7_Ny=7_Nz=1_Ne=8_0_0.csv")
in_vals     = 1000*raw_data['E0'].values
t_vals      = time_steps*10**6

with plt.style.context(['science','bright']):
    
    fig = plt.figure()
    fig.set_figwidth(2*single_col*cmeter)
    # fig.set_figheight(single_col*cmeter)
    ax1 = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)

    def update(row):

        ax1.clear()
        ax1.plot(t_vals, in_vals)
        ax1.plot(t_vals[row], in_vals[row], 'o')
        ax1.set_xlabel('$t$ [$\mu s$]')
        ax1.set_ylabel('Input [$mV$]')
        ax1.set_xlim(0,200)
        ax1.set_ylim(-50,50)

        ax2.clear()

        sizes       = np.repeat(0,49+8)
        s_vals      = states.iloc[row].values
        sizes[8:]   = 500*np.abs(s_vals)

        node_colors = np.repeat(blue_color, 49+8)
        node_colors[8:][s_vals < 0] = red_color

        nx.draw_networkx(G, pos=pos, node_size=sizes, ax=ax2, node_color=node_colors, arrows=False, labels=node_labels, font_size=8)
        ax.set_title(f'$t = {np.round((10**6)*time_steps[row],2)}$ $\mu s$')
        plt.tight_layout()
        # plt.axis('off')
        
    ani = animation.FuncAnimation(fig, update, frames=5, interval=200, repeat_delay=1000, repeat=True)
    ani.save("test.gif")

plt.show()